In [1]:
import string
import numpy as np

In [2]:
np.random.seed(1234)

In [3]:
!head robert_frost.txt

Two roads diverged in a yellow wood,
And sorry I could not travel both
And be one traveler, long I stood
And looked down one as far as I could
To where it bent in the undergrowth; 

Then took the other, as just as fair,
And having perhaps the better claim
Because it was grassy and wanted wear,
Though as for that the passing there


In [4]:
initial = {}
first_order = {}
second_order = {}

In [5]:
def add2dic(d, k, v):
    if k not in d:
        d[k] = []
    d[k].append(v)

In [6]:
with open('robert_frost.txt') as raw_data:
    for line in raw_data:
        line = line.rstrip().lower()
        line = line.translate(str.maketrans('', '', string.punctuation))
        tokens = line.split() #tokenization ->  list of token
        
        T = len(tokens)
        for i in range(T):
            t = tokens[i]
            if i == 0: #initial
                #measure the distribution of first order
                initial[t] = initial.get(t, 0.) + 1
            else:
                t_1 = tokens[i-1] #previous word
                if i == T-1:
                    #measure probability and ending the line
                    add2dic(second_order, (t_1, t), 'END') #fake token'END', notice the line is over
                if i == 1:
                    #measure distribution of second word
                    #given only the first word
                    add2dic(first_order, t_1, t)
                else:
                    t_2 = tokens[i-2]
                    add2dic(second_order, (t_2, t_1), t)

In [7]:
#normalize the distribution
initial_total = sum(initial.values()) #equal the number of lines counted

for t,c in initial.items():
    initial[t] = c/initial_total

In [8]:
#convert [cat, cat, cat, dog, dog,...mouse] into {cat:0.5,...}
def list2pdict(ts):
    d = {}
    n = len(ts)
    for t in ts:
        d[t] = d.get(t,0) + 1
    for t,c in d.items():
        d[t] = c/n
    return d

In [9]:
for t_1,ts in first_order.items():
    #replace list with dictionary of probabilities
    first_order[t_1] = list2pdict(ts)

In [10]:
for k, ts in second_order.items():
    second_order[k] = list2pdict(ts)

In [14]:
def sample_word(d):
    p0 = np.random.random() 
    cumulative = 0
    for t,p in d.items():
        cumulative += p
        if p0 < cumulative:
            return t
    assert(False)

In [16]:
#generate 4 lines of poem 
def generate():
    for i in range(4): # 4lines
        sentence = []
        
        #initial words
        w0 = sample_word(initial)
        sentence.append(w0)
        
        #sample second word
        w1 = sample_word(first_order[w0])
        sentence.append(w1)
        
        #second-order transitions until END
        while True:
            w2 = sample_word(second_order[(w0,w1)])
            if w2 == 'END':
                break
            sentence.append(w2)
            w0 = w1
            w1 = w2
        print(' '.join(sentence))

In [17]:
generate()

came over houses from another street
might just as empty
but it isnt as if and thats not all the money goes so fast
you couldnt call it living for it aint


In [18]:
generate()

one level higher than the earth
she has her speel on every single lizard
with whose vast wheels
its to say
